In [26]:
from compute_signals import *
from process_signal_data import *
from model_classifier import *
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import random
from scipy.interpolate import CubicSpline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from scipy.signal import welch
import os


- lf is the frequency band for the Low frequency from the paper
- hf is the frequency band for the High frequency from the paper
- percent_outliers is the fraction of values to remove from the lower and upper values of the window

- We will load the the dataframe that is provided and then subsequently add the feature columns to it

In [27]:
data = pd.read_csv('userData.csv')
# data.head()

In [28]:
lf =  [0.04, 0.15]
hf = [0.15, 0.4]
percent_outliers = 0.1
plot_flag = 1
save_fig = 1
kernel_type = 'poly'
window_size =  90
offset_window = 10
sample_points = 5


- **convert_rr_total** will convert the **rr** column to arrays and save it as a column in the dataframe
- **compute_sampling_rate** will compute the sampling rate for each sample and then save it as a column

In [29]:
convert_rr_total(data)
compute_sampling_rate(data)
data.head(15)

,user,age,gender,section,duration,arousal,valence,samples,rr,rr_array,fs
0,20230608_183012_3_02001743,42,0,Rest_Start,185,3,3,215,"[783, 790, 795, 769, 768, 754, 762, 766, 774, ...","[783.0, 790.0, 795.0, 769.0, 768.0, 754.0, 762...",1.162162
1,20230608_183012_3_02001743,42,0,Reading,185,4,2,207,"[739, 749, 724, 692, 701, 724, 757, 769, 777, ...","[739.0, 749.0, 724.0, 692.0, 701.0, 724.0, 757...",1.118919
2,20230608_183012_3_02001743,42,0,Breath_1,130,5,3,155,"[750, 751, 735, 741, 749, 760, 752, 761, 774, ...","[750.0, 751.0, 735.0, 741.0, 749.0, 760.0, 752...",1.192308
3,20230608_183012_3_02001743,42,0,Stroop,190,3,0,220,"[753, 765, 778, 757, 768, 774, 775, 773, 777, ...","[753.0, 765.0, 778.0, 757.0, 768.0, 774.0, 775...",1.157895
4,20230608_183012_3_02001743,42,0,ReactionTime,227,5,3,241,"[698, 741, 791, 814, 813, 827, 849, 870, 820, ...","[698.0, 741.0, 791.0, 814.0, 813.0, 827.0, 849...",1.061674
5,20230608_183012_3_02001743,42,0,Video+,179,5,3,177,"[780, 786, 785, 775, 770, 762, 749, 762, 769, ...","[780.0, 786.0, 785.0, 775.0, 770.0, 762.0, 749...",0.988827
6,20230608_183012_3_02001743,42,0,Breath_2,131,6,4,131,"[778, 773, 767, 761, 741, 749, 747, 758, 773, ...","[778.0, 773.0, 767.0, 761.0, 741.0, 749.0, 747...",1.000000
7,20230608_183012_3_02001743,42,0,Math,190,3,2,187,"[768, 761, 738, 723, 723, 712, 727, 724, 731, ...","[768.0, 761.0, 738.0, 723.0, 723.0, 712.0, 727...",0.984211
8,20230608_183012_3_02001743,42,0,ImageStroop,294,6,4,297,"[706, 706, 700, 716, 717, 743, 741, 737, 748, ...","[706.0, 706.0, 700.0, 716.0, 717.0, 743.0, 741...",1.010204
9,20230608_183012_3_02001743,42,0,Video-,162,3,1,187,"[720, 715, 732, 723, 708, 717, 719, 728, 703, ...","[720.0, 715.0, 732.0, 723.0, 708.0, 717.0, 719...",1.154321


In [30]:
users_pass = find_users(data)
users_unique =  data.user.unique()
len(users_unique)

40

In [31]:
dict_new =  {'user':users_unique,'section':None}
df_breath1 = pd.DataFrame.from_dict(dict_new)
df_breath2 = pd.DataFrame.from_dict(dict_new)
df_breath3 = pd.DataFrame.from_dict(dict_new)
df_breath4 = pd.DataFrame.from_dict(dict_new)

df_mental = pd.DataFrame.from_dict(dict_new)
df_rest_start = pd.DataFrame.from_dict(dict_new)
df_rest_end = pd.DataFrame.from_dict(dict_new)
df_stroop = pd.DataFrame.from_dict(dict_new)
df_imagestroop = pd.DataFrame.from_dict(dict_new)
df_math = pd.DataFrame.from_dict(dict_new)
df_video_plus = pd.DataFrame.from_dict(dict_new)
df_video_minus = pd.DataFrame.from_dict(dict_new)
df_breath_total = pd.DataFrame.from_dict(dict_new)
df_rest_total  = pd.DataFrame.from_dict(dict_new)
df_reaction_time  = pd.DataFrame.from_dict(dict_new)
df_reading  = pd.DataFrame.from_dict(dict_new)

df_exercise = pd.DataFrame.from_dict(dict_new)
# a step to make empty dataframes for each user and each section


In [32]:
df_video_minus.head()

,user,section
0,20230608_183012_3_02001743,None
1,20230608_210755_4_02001743,None
2,20230609_180159_5_02001743,None
3,20230609_230133_6_02001743,None
4,20230613_173426_9_02001743,None


- **catx_vals** is the extracted signal containg each category
- Each dataframe then has a column called **signal** that has each extracted signal

In [33]:
stroop_vals = construct_signal_general(data,users_unique,['Stroop'])
math_vals = construct_signal_general(data,users_unique,['Math'])
video_minus_vals = construct_signal_general(data,users_unique,['Video-'])
video_plus_vals = construct_signal_general(data,users_unique,['Video+'])
imagestroop_vals = construct_signal_general(data,users_unique,['ImageStroop'])
breath1_vals = construct_signal_general(data,users_unique,['Breath_1'])
breath2_vals = construct_signal_general(data,users_unique,['Breath_2'])
breath3_vals = construct_signal_general(data,users_unique,['Breath_3'])
breath4_vals = construct_signal_general(data,users_unique,['Breath_4'])
breath_total_val = construct_signal_general(data,users_unique,['Breath_1','Breath_2','Breath_3', 'Breath_4'])
rest_start_vals = construct_signal_general(data,users_unique,['Rest_Start'])
rest_end_vals = construct_signal_general(data,users_unique,['Rest_End'])
rest_total_vals = construct_signal_general(data,users_unique,['Rest_Start','Rest_End'])
reactiontime_vals = construct_signal_general(data,users_unique,['ReactionTime'])
reading_vals = construct_signal_general(data,users_unique,['Reading'])


df_breath1['signal'] = breath1_vals
df_breath2['signal'] = breath2_vals
df_breath3['signal'] = breath3_vals
df_breath4['signal'] = breath4_vals
df_breath_total['signal'] = breath_total_val

df_rest_start['signal'] = rest_start_vals
df_rest_end['signal'] = rest_end_vals
df_rest_total['signal'] = rest_total_vals
df_stroop['signal'] = stroop_vals
df_imagestroop['signal'] = imagestroop_vals
df_math['signal'] = math_vals
df_video_plus['signal'] = video_plus_vals
df_video_minus['signal'] = video_minus_vals
df_reaction_time['signal'] = reactiontime_vals
df_reading['signal'] = reading_vals


# i feel like this is being done to make separate dataframes for each activity which is lost to me at the moment - so lets see 

In [34]:
df_video_minus.head()

,user,section,signal
0,20230608_183012_3_02001743,None,"[720.0, 715.0, 732.0, 723.0, 708.0, 717.0, 719..."
1,20230608_210755_4_02001743,None,"[752.0, 727.0, 662.0, 642.0, 712.0, 765.0, 815..."
2,20230609_180159_5_02001743,None,"[747.0, 775.0, 800.0, 751.0, 706.0, 729.0, 739..."
3,20230609_230133_6_02001743,None,"[731.0, 735.0, 758.0, 736.0, 747.0, 747.0, 760..."
4,20230613_173426_9_02001743,None,"[668.0, 668.0, 660.0, 651.0, 651.0, 643.0, 640..."


In [35]:
stroop_fs,stroop_samples, stroop_duration = construct_fs_general(data,users_unique,['Stroop'])
math_fs,math_samples, math_duration = construct_fs_general(data,users_unique,['Math'])
videoplus_fs,videoplus_samples, videoplus_duration = construct_fs_general(data,users_unique,['Video+'])
videominus_fs,videominus_samples, videominus_duration = construct_fs_general(data,users_unique,['Video-'])
imagestroop_fs,imagestroop_samples, imagestroop_duration = construct_fs_general(data,users_unique,['ImageStroop'])
breath1_fs,breath1_samples, breath1_duration = construct_fs_general(data,users_unique,['Breath_1'])
breath2_fs,breath2_samples, breath2_duration = construct_fs_general(data,users_unique,['Breath_2'])
breath3_fs,breath3_samples, breath3_duration = construct_fs_general(data,users_unique,['Breath_3'])
breath4_fs,breath4_samples, breath4_duration = construct_fs_general(data,users_unique,['Breath_4'])
breathtotal_fs,breathtotal_samples, breathtotal_duration = construct_fs_general(data,users_unique, ['Breath_1','Breath_2','Breath_3', 'Breath_4'])

rest_start_fs,rest_start_samples, rest_start_duration = construct_fs_general(data,users_unique,['Rest_Start'])
rest_end_fs,rest_end_samples, rest_end_duration= construct_fs_general(data,users_unique,['Rest_End'])
rest_total_fs,rest_total_samples, rest_total_duration= construct_fs_general(data, users_unique,['Rest_Start','Rest_End'])
reaction_fs,reaction_samples, reaction_duration= construct_fs_general(data, users_unique,['ReactionTime'])
reading_fs,reading_samples, reading_duration= construct_fs_general(data, users_unique,['Reading'])


In [36]:
print(videominus_fs)
print(type(videominus_fs[0]))

[array([1.15432099]), array([1.09876543]), array([1.02453988]), array([1.17901235]), array([1.36196319]), array([0.67484663]), array([1.0308642]), array([1.08588957]), array([1.2345679]), array([0.96319018]), array([0.93865031]), array([1.06748466]), array([0.87730061]), array([0.99386503]), array([0.86503067]), array([1.1595092]), array([0.69135802]), array([0.85185185]), array([0.95705521]), array([0.58282209]), array([0.83950617]), array([0.99386503]), array([0.87730061]), array([0.94444444]), array([1.2208589]), array([1.34969325]), array([0.84662577]), array([1.01234568]), array([0.81595092]), array([1.09259259]), array([1.11042945]), array([1.03680982]), array([0.73006135]), array([1.42331288]), array([0.86419753]), array([0.96932515]), array([0.99382716]), array([0.95092025]), array([0.88271605]), array([1.09259259])]
<class 'numpy.ndarray'>


In [37]:
print(videominus_samples)

[array([187]), array([178]), array([167]), array([191]), array([222]), array([110]), array([167]), array([177]), array([200]), array([157]), array([153]), array([174]), array([143]), array([162]), array([141]), array([189]), array([112]), array([138]), array([156]), array([95]), array([136]), array([162]), array([143]), array([153]), array([199]), array([220]), array([138]), array([164]), array([133]), array([177]), array([181]), array([169]), array([119]), array([232]), array([140]), array([158]), array([161]), array([155]), array([143]), array([177])]


In [38]:
df_breath1['fs'] = np.array(breath1_fs)
df_breath2['fs'] = np.array(breath2_fs)
df_breath3['fs'] = np.array(breath3_fs)
df_breath4['fs'] = np.array(breath4_fs)
df_breath_total['fs'] = np.array(breathtotal_fs)


df_rest_start['fs'] = np.array(rest_start_fs)
df_rest_end['fs'] = np.array(rest_end_fs)
df_rest_total['fs'] = np.array(rest_total_fs)

df_stroop['fs'] = np.array(stroop_fs)
df_imagestroop['fs'] = np.array(imagestroop_fs)
df_math['fs'] = np.array(math_fs)
df_video_plus['fs'] = np.array(videoplus_fs)
df_video_minus['fs'] = np.array(videominus_fs)
df_reaction_time['fs'] = np.array(reaction_fs)
df_reading['fs'] = np.array(reading_fs)


In [39]:
df_video_minus.head()

,user,section,signal,fs
0,20230608_183012_3_02001743,None,"[720.0, 715.0, 732.0, 723.0, 708.0, 717.0, 719...",1.154321
1,20230608_210755_4_02001743,None,"[752.0, 727.0, 662.0, 642.0, 712.0, 765.0, 815...",1.098765
2,20230609_180159_5_02001743,None,"[747.0, 775.0, 800.0, 751.0, 706.0, 729.0, 739...",1.024540
3,20230609_230133_6_02001743,None,"[731.0, 735.0, 758.0, 736.0, 747.0, 747.0, 760...",1.179012
4,20230613_173426_9_02001743,None,"[668.0, 668.0, 660.0, 651.0, 651.0, 643.0, 640...",1.361963


In [40]:
df_breath1['samples'] = np.array(breath1_samples)
df_breath2['samples'] = np.array(breath2_samples)
df_breath3['samples'] = np.array(breath3_samples)
df_breath4['samples'] = np.array(breath4_samples)
df_breath_total['samples'] = np.array(breathtotal_samples)


df_rest_start['samples'] = np.array(rest_start_samples)
df_rest_end['samples'] = np.array(rest_end_samples)
df_rest_total['samples'] = np.array(rest_total_samples)

df_stroop['samples'] = np.array(stroop_samples)
df_imagestroop['samples'] = np.array(imagestroop_samples)
df_math['samples'] = np.array(math_samples)
df_video_plus['samples'] = np.array(videoplus_samples)
df_video_minus['samples'] = np.array(videominus_samples)
df_reaction_time['samples'] = np.array(reaction_samples)
df_reading['samples'] = np.array(reading_samples)

In [41]:
df_video_minus.head()

,user,section,signal,fs,samples
0,20230608_183012_3_02001743,None,"[720.0, 715.0, 732.0, 723.0, 708.0, 717.0, 719...",1.154321,187
1,20230608_210755_4_02001743,None,"[752.0, 727.0, 662.0, 642.0, 712.0, 765.0, 815...",1.098765,178
2,20230609_180159_5_02001743,None,"[747.0, 775.0, 800.0, 751.0, 706.0, 729.0, 739...",1.024540,167
3,20230609_230133_6_02001743,None,"[731.0, 735.0, 758.0, 736.0, 747.0, 747.0, 760...",1.179012,191
4,20230613_173426_9_02001743,None,"[668.0, 668.0, 660.0, 651.0, 651.0, 643.0, 640...",1.361963,222


In [42]:
df_breath1['duration'] = np.array(breath1_duration)
df_breath2['duration'] = np.array(breath2_duration)
df_breath3['duration'] = np.array(breath3_duration)
df_breath4['duration'] = np.array(breath4_duration)
df_breath_total['duration'] = np.array(breathtotal_duration)

df_rest_start['duration'] = np.array(rest_start_duration)
df_rest_end['duration'] = np.array(rest_end_duration)
df_rest_total['duration'] = np.array(rest_total_duration)

df_stroop['duration'] = np.array(stroop_duration)
df_imagestroop['duration'] = np.array(imagestroop_duration)
df_math['duration'] = np.array(math_duration)
df_video_plus['duration'] = np.array(videoplus_duration)
df_video_minus['duration'] = np.array(videominus_duration)
df_reaction_time['duration'] = np.array(reaction_duration)
df_reading['duration'] = np.array(reading_duration)

In [43]:
df_video_minus.head()

,user,section,signal,fs,samples,duration
0,20230608_183012_3_02001743,None,"[720.0, 715.0, 732.0, 723.0, 708.0, 717.0, 719...",1.154321,187,162
1,20230608_210755_4_02001743,None,"[752.0, 727.0, 662.0, 642.0, 712.0, 765.0, 815...",1.098765,178,162
2,20230609_180159_5_02001743,None,"[747.0, 775.0, 800.0, 751.0, 706.0, 729.0, 739...",1.024540,167,163
3,20230609_230133_6_02001743,None,"[731.0, 735.0, 758.0, 736.0, 747.0, 747.0, 760...",1.179012,191,162
4,20230613_173426_9_02001743,None,"[668.0, 668.0, 660.0, 651.0, 651.0, 643.0, 640...",1.361963,222,163


In [44]:
# combing all the dataframes into one big dataframe from this point onwards i will be writing my own code and not using the one provided originally
# List of your dataframes and their corresponding activities
dataframes = {
    "breath_total": df_breath_total,
    "rest_total": df_rest_total,
    "stroop": df_stroop,
    "imagestroop": df_imagestroop,
    "math": df_math,
    "video_plus": df_video_plus,
    "video_minus": df_video_minus,
    "reaction_time": df_reaction_time,
    "reading": df_reading
}

# Adding the 'activity' column to each dataframe
for activity, df in dataframes.items():
    df['activity'] = activity

# Concatenating all dataframes
main_df = pd.concat(dataframes.values(), ignore_index=True)
main_df.head()

#drop column section in the main dataframe
main_df = main_df.drop(columns=['section'])
main_df.head()

,user,signal,fs,samples,duration,activity
0,20230608_183012_3_02001743,"[750.0, 751.0, 735.0, 741.0, 749.0, 760.0, 752...",0.928983,484,521,breath_total
1,20230608_210755_4_02001743,"[712.0, 716.0, 729.0, 681.0, 668.0, 672.0, 628...",1.126923,586,520,breath_total
2,20230609_180159_5_02001743,"[631.0, 638.0, 655.0, 731.0, 681.0, 676.0, 715...",1.177820,616,523,breath_total
3,20230609_230133_6_02001743,"[719.0, 719.0, 720.0, 731.0, 701.0, 711.0, 706...",1.024952,534,521,breath_total
4,20230613_173426_9_02001743,"[629.0, 631.0, 627.0, 634.0, 626.0, 630.0, 625...",1.262452,659,522,breath_total


In [45]:
type(main_df['signal'][0])

numpy.ndarray

In [46]:
#main_df.to_csv('main_df.csv', index=False)


In [53]:
# Computing hf and lf ussing fft and not the butter_band pass filter used in the paper redefinig the window as mentioned in the paper 
# Function to compute LF, HF, and LF/HF ratio
def compute_hrv_metrics(rr_intervals, fs):
    # Calculate power spectral density using Welch's method
    f, pxx = welch(rr_intervals, fs=fs, nperseg=len(rr_intervals))

    # Frequency bands for LF and HF components
    lf_band = (0.04, 0.15)
    hf_band = (0.15, 0.4)

    # Calculate power in LF and HF bands
    lf_power = np.trapz(pxx[(f >= lf_band[0]) & (f <= lf_band[1])], f[(f >= lf_band[0]) & (f <= lf_band[1])])
    hf_power = np.trapz(pxx[(f >= hf_band[0]) & (f <= hf_band[1])], f[(f >= hf_band[0]) & (f <= hf_band[1])])

    # Calculate LF/HF ratio
    lf_hf_ratio = lf_power / hf_power

    return lf_power, hf_power, lf_hf_ratio

# Sliding window function
def sliding_window_analysis(rr_intervals, fs, window_length_sec=120, increment_sec=10): # window length is 2 minutes and increment is 10 seconds
    window_samples = int(window_length_sec * fs)
    increment_samples = int(increment_sec * fs)
    n_samples = len(rr_intervals)

    if n_samples < window_samples:  # If the signal is shorter than the window size
        return compute_hrv_metrics(rr_intervals, fs)  # Compute metrics for the entire signal

    # Calculate the number of windows
    n_windows = int(np.floor((n_samples - window_samples) / increment_samples) + 1)

    # Lists to store results for each window
    lf_values = []
    hf_values = []
    lf_hf_ratio_values = []

    # Slide the window and compute metrics
    for i in range(n_windows):
        start_idx = i * increment_samples
        end_idx = start_idx + window_samples
        windowed_rr = rr_intervals[start_idx:end_idx]

        lf, hf, lf_hf = compute_hrv_metrics(windowed_rr, fs)
        lf_values.append(lf)
        hf_values.append(hf)
        lf_hf_ratio_values.append(lf_hf)

    return lf_values, hf_values, lf_hf_ratio_values

In [54]:
# Function to process each row in the dataframe
def process_row(row):
    rr_intervals = row['signal']
    fs = row['fs']
    lf, hf, lf_hf = sliding_window_analysis(rr_intervals, fs)
    return pd.Series([lf, hf, lf_hf])


In [57]:
# updating the main_df
main_df[['LF', 'HF', 'LF_HF']] = main_df.apply(process_row, axis=1)


In [59]:
main_df[50:100]

,user,signal,fs,samples,duration,activity,LF,HF,LF_HF
50,20230617_174750_4_02001754,"[661.0, 666.0, 670.0, 682.0, 688.0, 683.0, 678...",0.897297,332,370,rest_total,"[7645.7754725257255, 8038.361370010089, 7901.1...","[16056.909966434074, 21510.539581646586, 25314...","[0.4761673004649538, 0.37369408328876375, 0.31..."
51,20230617_183827_15_02001743,"[798.0, 771.0, 800.0, 826.0, 829.0, 825.0, 838...",1.143243,423,370,rest_total,"[3421.499909983005, 4026.0586016131606, 4640.7...","[592.6607907636585, 585.622262627987, 581.5793...","[5.773116702345562, 6.8748387118108765, 7.9796..."
52,20230617_204444_5_02001754,"[1030.0, 996.0, 1022.0, 1152.0, 1025.0, 968.0,...",0.808108,299,370,rest_total,"[187.79768202828635, 142.5538438598839, 135.24...","[1501.598438467728, 1370.9805219228617, 1159.7...","[0.12506518202024786, 0.10397948153190807, 0.1..."
53,20230618_174151_16_02001743,"[708.0, 721.0, 735.0, 746.0, 720.0, 751.0, 771...",1.077957,401,372,rest_total,"[482.95744444615536, 548.9023176153672, 677.64...","[545.0309438502128, 1989.2583522333484, 4752.3...","[0.8861101372234809, 0.2759331471445689, 0.142..."
54,20230618_174513_3_02001745,"[802.0, 785.0, 762.0, 764.0, 762.0, 772.0, 780...",0.867568,321,370,rest_total,"[786.2763903937146, 709.9851180640693, 899.836...","[200.00908903767558, 183.33020234222698, 256.4...","[3.9312032976941573, 3.8727122372271356, 3.508..."
55,20230618_223836_6_02001754,"[750.0, 750.0, 768.0, 759.0, 759.0, 785.0, 806...",1.045946,387,370,rest_total,"[214.03923480457445, 182.07236114095315, 169.3...","[143.08227290508952, 148.1323487728867, 170.88...","[1.4959172122360165, 1.2291195181148618, 0.990..."
56,20230619_174100_17_02001743,"[1077.0, 1123.0, 1085.0, 1103.0, 1084.0, 1130....",0.694595,257,370,rest_total,"[34.31201367045416, 33.90389866677955, 51.4399...","[149.2965039691111, 175.59079807652492, 241.02...","[0.22982462923279964, 0.1930847119448922, 0.21..."
57,20230619_175414_4_02001745,"[860.0, 889.0, 888.0, 856.0, 811.0, 834.0, 909...",0.886486,328,370,rest_total,"[308.7826339192739, 280.8016295651427, 282.517...","[196.48475123721283, 206.4117959441984, 231.16...","[1.5715348492692223, 1.3603952636556438, 1.222..."
58,20230621_193606_7_02001754,"[679.0, 735.0, 809.0, 859.0, 810.0, 860.0, 842...",0.905405,335,370,rest_total,"[8208.12542837385, 6731.520272201991, 5319.048...","[15285.929635353405, 12757.204052132964, 9726....","[0.5369726031833904, 0.5276642314956547, 0.546..."
59,20230621_194319_5_02001745,"[701.0, 711.0, 736.0, 815.0, 765.0, 770.0, 787...",0.748649,277,370,rest_total,"[1468.3517240910526, 1493.4477155723225, 1609....","[1528.2622130052757, 989.2904037710645, 962.15...","[0.9607982920703043, 1.5096150835785598, 1.672..."


In [ ]:
# main_df.to_csv('main_df_LF_HF.csv', index=False)

In [61]:
len(main_df['LF'][300])

18

In [62]:
# Making new dataframe with just the user and activity columns and lf_hf columns will use this later for the svm approach and comparison 
df_P1_lf_hf = main_df[['user', 'activity', 'LF_HF']]
df_P1_lf_hf.head()
# print(df_P1_lf_hf.shape)

,user,activity,LF_HF
0,20230608_183012_3_02001743,breath_total,"[4.578940708636031, 5.039292189476101, 6.30402..."
1,20230608_210755_4_02001743,breath_total,"[1.247832100030658, 0.8930347441208322, 0.9557..."
2,20230609_180159_5_02001743,breath_total,"[6.024086186862033, 6.429796170979665, 6.06763..."
3,20230609_230133_6_02001743,breath_total,"[0.5551263783555743, 0.5742567524978311, 0.570..."
4,20230613_173426_9_02001743,breath_total,"[0.44626080967528686, 0.4456185823002845, 0.47..."


In [63]:
main_df.shape

(360, 9)